In [2]:
import psycopg2
import pandas as pd
from credentials import PW

In [2]:
conn = psycopg2.connect(
    dbname = "postgres", 
    user = "postgres", 
    password = PW, 
    host = "127.0.0.1", 
    port = "5433")

conn.set_session(autocommit=True)
cur = conn.cursor()
cur.execute("CREATE DATABASE climb")
conn.close()

In [3]:
conn = psycopg2.connect(
    dbname = "climb", 
    user = "postgres", 
    password = PW, 
    host = "127.0.0.1", 
    port = "5433")

cur = conn.cursor()

In [4]:
def drop_tables(cur, conn):
    for query in drop_tables_queries:
        cur.execute(query)
        conn.commit()

In [5]:
def create_tables(cur, conn):
    for query in create_tables_queries:
        cur.execute(query)
        conn.commit()

In [6]:
routes = pd.read_csv("data/mp_routes.csv", index_col=0)

In [7]:
routes.head()

,Route,Location,URL,Avg Stars,Route Type,Rating,Pitches,Length,Area Latitude,Area Longitude,desc,protection,num_votes
0,Access Denied,El Mirador > El Potrero Chico > Nuevo Leon > N...,https://www.mountainproject.com/route/11014983...,2.9,Sport,5.10b/c,4,350.0,25.95044,-100.47755,This is a really great route~ with awesome exp...,12 draws + 60m Rope Take 22 draws if you wan...,22
1,Agave Nectar,Sugar Shack > Cougar Canyon (Creek) - CONSTRUC...,https://www.mountainproject.com/route/11091386...,2.0,Sport,5.10b/c,1,NaN,51.09642,-115.31767,from tabvar: Cool fins to roof~ thin holds...,4 bolts to anchor,1
2,Ant & Bee do Yoga,The Hen House > Kamloops > British Columbia > ...,https://www.mountainproject.com/route/11240652...,2.7,Trad,5.10b/c,1,NaN,50.57212,-120.13874,A safe mixed route with a bit of run out up to...,"mixed~ gear to 4""",3
3,Besame Fuerte,Pilon De Lolita > Loreto Area > Baja Californi...,https://www.mountainproject.com/route/11608640...,2.0,Sport,5.10b/c,1,80.0,26.01097,-111.34166,Start on a slab under a left leaning arched ro...,bolts,1
4,Big Momma's Rock,The Courtyard > Mamquam FSR > Squamish > Briti...,https://www.mountainproject.com/route/11445772...,3.0,Sport,5.10b/c,1,60.0,49.71393,-123.09943,Fun technical climbing. Tricky right off the bat.,bolts,3


In [8]:
routes.shape

(116700, 13)

In [9]:
routes.columns

Index(['Route', 'Location', 'URL', 'Avg Stars', 'Route Type', 'Rating',
       'Pitches', 'Length', 'Area Latitude', 'Area Longitude', ' desc',
       ' protection', ' num_votes'],
      dtype='object')

In [10]:
routes = routes.rename(columns={
    ' desc': 'Description',
    ' protection': 'Protection',
    ' num_votes': 'Votes'
})

In [11]:
routes.head(1)

,Route,Location,URL,Avg Stars,Route Type,Rating,Pitches,Length,Area Latitude,Area Longitude,Description,Protection,Votes
0,Access Denied,El Mirador > El Potrero Chico > Nuevo Leon > N...,https://www.mountainproject.com/route/11014983...,2.9,Sport,5.10b/c,4,350.0,25.95044,-100.47755,This is a really great route~ with awesome exp...,12 draws + 60m Rope Take 22 draws if you wan...,22


In [12]:
routes_table_create = """
CREATE TABLE routes (
    route_id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    type VARCHAR(50),
    rating VARCHAR(50),
    pitches INT,
    length FLOAT,
    description TEXT,
    protection TEXT,
    avg_stars FLOAT,
    num_votes INT
);
"""

In [13]:
locations_table_create = """
CREATE TABLE locations (
    location_id SERIAL PRIMARY KEY,
    route_id INT REFERENCES routes(route_id),
    detailed_location TEXT,
    area_latitude FLOAT,
    area_longitude FLOAT
);
"""

In [14]:
urls_table_create = """
CREATE TABLE urls (
    url_id SERIAL PRIMARY KEY,
    route_id INT REFERENCES routes(route_id),
    url TEXT
);
"""

In [15]:
cur = conn.cursor()
cur.execute(routes_table_create)
cur.execute(locations_table_create)
cur.execute(urls_table_create)
conn.commit()

In [16]:
try:
    
    for idx, row in routes.iterrows():
        cur.execute(
            "INSERT INTO routes (name, type, rating, pitches, length, description, protection, avg_stars, num_votes) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING route_id",
            (row['Route'], row['Route Type'], row['Rating'], row['Pitches'], row['Length'], row['Description'], row['Protection'], row['Avg Stars'], row['Votes'])
        )
        route_id = cur.fetchone()[0]
        
        # Insert into locations
        cur.execute(
            "INSERT INTO locations (route_id, detailed_location, area_latitude, area_longitude) VALUES (%s, %s, %s, %s)",
            (route_id, row['Location'], row['Area Latitude'], row['Area Longitude'])
        )

        # Insert into urls
        cur.execute(
            "INSERT INTO urls (route_id, url) VALUES (%s, %s)",
            (route_id, row['URL'])
        )

    conn.commit()
except Exception as e:
    print("An error occurred:", e)
    conn.rollback()
finally:
    cur.close()
    conn.close()

#### Routes Table: (Primary)
Columns: Route ID (primary key), Route Name, Route Type, Rating, Pitches, Length, Description, Protection, Average Stars, Number of Votes.
The 'Route ID' can be a unique identifier for each route.

#### Locations Table:
Columns: Location ID (primary key), Route ID (foreign key), Detailed Location, Area Latitude, Area Longitude.
'Location ID' is a unique identifier for each location. 'Route ID' links back to the Routes table.


#### URLs Table:
Columns: URL ID (primary key), Route ID (foreign key), URL.
'URL ID' is a unique identifier for each URL.

In [3]:
conn = psycopg2.connect(
    dbname = "climb", 
    user = "postgres", 
    password = PW, 
    host = "127.0.0.1", 
    port = "5433")

In [13]:
routes_query = "SELECT * FROM routes LIMIT 10;"
routes_table = pd.read_sql(routes_query, conn, index_col = 'route_id')
routes_table.head(10)

/var/folders/zr/_py5mgyx41dgfml7lmxyyff80000gn/T/ipykernel_46701/4086977808.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  routes_table = pd.read_sql(routes_query, conn, index_col = 'route_id')


,name,type,rating,pitches,length,description,protection,avg_stars,num_votes
route_id,,,,,,,,,
1,Access Denied,Sport,5.10b/c,4,350.0,This is a really great route~ with awesome exp...,12 draws + 60m Rope Take 22 draws if you wan...,2.9,22
2,Agave Nectar,Sport,5.10b/c,1,NaN,from tabvar: Cool fins to roof~ thin holds...,4 bolts to anchor,2.0,1
3,Ant & Bee do Yoga,Trad,5.10b/c,1,NaN,A safe mixed route with a bit of run out up to...,"mixed~ gear to 4""",2.7,3
4,Besame Fuerte,Sport,5.10b/c,1,80.0,Start on a slab under a left leaning arched ro...,bolts,2.0,1
5,Big Momma's Rock,Sport,5.10b/c,1,60.0,Fun technical climbing. Tricky right off the bat.,bolts,3.0,3
6,Big Splash,Sport,5.10b/c,1,50.0,Two potential starts. 1) Travers across canyon...,Free solo. Caution~ control your fall~ water c...,3.0,1
7,Black Smear,Trad,5.10b/c X,1,20.0,On the upper slabs there is a prominent black ...,none~ solo or TR.,2.0,1
8,Brace yourself Sheila,Sport,5.10b/c,1,NaN,If you miss the high left hold at crux~ the ro...,7 bolts~ chain anchor.,2.0,1
9,Cabot Crack,Sport,5.10b/c,1,75.0,Climbs a crack.,Bolts,3.0,1


In [14]:
locations_query = "SELECT * FROM locations LIMIT 10;"
locations_table = pd.read_sql(locations_query, conn, index_col = 'route_id')
locations_table.head(10)

/var/folders/zr/_py5mgyx41dgfml7lmxyyff80000gn/T/ipykernel_46701/1255866853.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  locations_table = pd.read_sql(locations_query, conn, index_col = 'route_id')


,location_id,detailed_location,area_latitude,area_longitude
route_id,,,,
1,1,El Mirador > El Potrero Chico > Nuevo Leon > N...,25.95044,-100.47755
2,2,Sugar Shack > Cougar Canyon (Creek) - CONSTRUC...,51.09642,-115.31767
3,3,The Hen House > Kamloops > British Columbia > ...,50.57212,-120.13874
4,4,Pilon De Lolita > Loreto Area > Baja Californi...,26.01097,-111.34166
5,5,The Courtyard > Mamquam FSR > Squamish > Briti...,49.71393,-123.09943
6,6,The Dark Pool > Powell River > British Columbi...,49.84860,-124.52870
7,7,The Slabs > Bear Mountain (AKA Bald Hill) > Ne...,45.47227,-66.50856
8,8,(a) Foreplay Left and Centre > Foreplay Cliff ...,49.45248,-119.56587
9,9,Los Chileros > San Isidro Canyon > Nuevo Leon ...,25.37841,-100.31211


In [15]:
urls_query = "SELECT * FROM urls LIMIT 10;"
urls_table = pd.read_sql(urls_query, conn, index_col = 'route_id')
urls_table.head(10)

/var/folders/zr/_py5mgyx41dgfml7lmxyyff80000gn/T/ipykernel_46701/1038380517.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  urls_table = pd.read_sql(urls_query, conn, index_col = 'route_id')


,url_id,url
route_id,,
1,1,https://www.mountainproject.com/route/11014983...
2,2,https://www.mountainproject.com/route/11091386...
3,3,https://www.mountainproject.com/route/11240652...
4,4,https://www.mountainproject.com/route/11608640...
5,5,https://www.mountainproject.com/route/11445772...
6,6,https://www.mountainproject.com/route/10744643...
7,7,https://www.mountainproject.com/route/10746663...
8,8,https://www.mountainproject.com/route/11228843...
9,9,https://www.mountainproject.com/route/11637745...
